In [1]:
input_dir = "./dataset"
output_dir = "./output"

In [2]:
import numpy as np
import torch
import time
import os
import torch.multiprocessing as mp
import train
import pandas as pd
                
max_epochs = 5
gpu_count = 2

os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '8888'

start_time = time.time()
mp.spawn(train.train, nprocs=gpu_count, args=(gpu_count,max_epochs,input_dir,))   
end_time = time.time() - start_time
print(end_time)

Epoch: 1
Loader: train, accuracy 0.8426785714285714, loss: 1.0623046228573436
Loader: valid, accuracy 0.9669047619047619, loss: 0.10423148078499017
Epoch: 2
Loader: train, accuracy 0.9709523809523809, loss: 0.09306491796104681
Loader: valid, accuracy 0.9714285714285714, loss: 0.08670391382097646
Epoch: 3
Loader: train, accuracy 0.9820238095238095, loss: 0.05858581774823722
Loader: valid, accuracy 0.9821428571428571, loss: 0.054981394187995684
Epoch: 4
Loader: train, accuracy 0.9866666666666667, loss: 0.03981670409973179
Loader: valid, accuracy 0.9845238095238096, loss: 0.04563605464580986
Epoch: 5
Loader: train, accuracy 0.9888690476190476, loss: 0.032789303371239274
Loader: valid, accuracy 0.9885714285714285, loss: 0.03391613965612595
24.19433617591858


In [3]:
test_data = pd.read_csv("./dataset/test.csv")
test_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from dataset import TestDataset
from transforms import data_tfs

test_dataset = TestDataset(test_data.to_numpy().astype(np.float32), transform=data_tfs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

In [5]:
result = []

model.eval()
for x_batch in test_loader:
    with torch.no_grad():
        outp = model(x_batch)
        preds = outp.argmax(1)
        result += preds.tolist()

NameError: name 'model' is not defined

In [ ]:
submission = pd.read_csv(f"{input_dir}/sample_submission.csv")
submission.head()

In [ ]:
submission['Label'] = result
submission

In [ ]:
submission.to_csv(f"{output_dir}/submission.csv", index=None)